In [1]:
%pip install -q python-terrier pyterrier_pisa 
%pip install -q git+https://github.com/naver/splade.git git+https://github.com/cmacdonald/pyt_splade.git

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pyterrier as pt
import pandas as pd
import pyt_splade
import torch

In [2]:
ds_topics = pt.get_dataset('irds:msmarco-passage/trec-dl-2019/judged')
ds_text = pt.get_dataset('irds:msmarco-passage')
topics = ds_topics.get_topics()

Java started (triggered by _pt_tokeniser) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]


In [3]:
import os
# Set environment variable for offline mode
os.environ['TRANSFORMERS_OFFLINE'] = '1'

# Ensure the local model path exists
local_model_path="/home/killdollar/btp/Information-Retrieval/splade-model"
if not os.path.exists(local_model_path):
	raise FileNotFoundError(f"The path {local_model_path} does not exist. Please ensure the model is downloaded.")

# Initialize SPLADE with explicit parameters# Force CPU usage
# Add this code at the beginning of your notebook (after imports)
import warnings
import torch
from contextlib import contextmanager

# Suppress specific warnings about CUDA autocast deprecation
warnings.filterwarnings("ignore", message=".*torch.cuda.amp.autocast.*")
warnings.filterwarnings("ignore", message=".*User provided device_type of 'cuda', but CUDA is not available.*")

# Create a silent replacement for torch.cuda.amp.autocast
@contextmanager
def silent_autocast():
    """Silent replacement for torch.cuda.amp.autocast that does nothing"""
    yield

# Monkey patch torch.cuda.amp.autocast to suppress warnings
torch.cuda.amp.autocast = silent_autocast

# Continue using CPU settings
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Hide all GPUs
torch.set_num_threads(4)  # Limit CPU threads to
# Initialize SPLADE with explicit CPU parameters
splade = pyt_splade.Splade(
    model=local_model_path,
    device="cpu",  # Force CPU
    max_length=128  # Reduce token length to save memory
)

/home/killdollar/btp/Information-Retrieval/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
indexer = pt.IterDictIndexer('./indices/msmarco-passage')
index = indexer.path
bm25 = pt.terrier.Retriever(index, wmodel="BM25", num_results=100)
splade_reranker = splade.scorer()

In [5]:
def get_text_for_docnos(docnos, dataset):
    """Efficiently get text for specific document IDs"""
    doc_texts = {}
    
    # Convert all docnos to the format we'll try
    docno_variants = {}
    for docno in docnos:
        str_docno = str(docno)
        docno_variants[str_docno] = docno
        # Also try integer version if it's numeric
        if str_docno.isdigit():
            docno_variants[int(str_docno)] = docno
    
    # Only iterate through corpus until we find all needed documents
    needed_ids = set(docno_variants.keys())
    found_count = 0
    
    corpus_iter = dataset.get_corpus_iter()
    for doc in corpus_iter:
        doc_id = doc['docno']
        
        # Check if this document is one we need
        if doc_id in needed_ids:
            original_docno = docno_variants[doc_id]
            doc_texts[original_docno] = doc['text']
            found_count += 1
            
            # Stop early if we found all documents we need
            if found_count >= len(docnos):
                break
    
    # Fill in missing documents
    for docno in docnos:
        if docno not in doc_texts:
            doc_texts[docno] = "Document text not available"
    
    return doc_texts

In [6]:
import gc

# Replace your current query processing loop with this optimized version
all_results = []

for idx, row in topics.iterrows():
    query_id = row['qid']
    query_text = row['query']
    
    print(f"Processing query {idx+1}/{len(topics)}: {query_text}")
    
    try:
        # Step 1: Get initial results from BM25
        initial_results = bm25.search(query_text)
        
        # Step 2: Process in smaller batches to avoid memory issues
        batch_size = 20  # Process 20 documents at a time
        all_reranked = []
        
        for start_idx in range(0, len(initial_results), batch_size):
            # Take a batch
            batch = initial_results.iloc[start_idx:start_idx+batch_size].copy()
            
            # Get text for this batch only
            docnos = batch['docno'].tolist()
            doc_texts = get_text_for_docnos(docnos, ds_text)
            batch['text'] = batch['docno'].map(doc_texts)
            batch['query'] = query_text
            
            # Apply SPLADE scoring to this batch
            reranked_batch = splade_reranker.transform(batch)
            all_reranked.append(reranked_batch)
            
            # Clean memory after each batch
            gc.collect()
            
        # Combine all batches
        if all_reranked:
            reranked_results = pd.concat(all_reranked)
            # Sort by SPLADE score and take top 10
            top_results = reranked_results.sort_values('score', ascending=False).head(10)
            print(f"SPLADE reranking successful")
        else:
            # Fallback to BM25 if all batches failed
            top_results = initial_results.head(10)
            top_results['text'] = top_results['docno'].map(lambda d: get_text_for_docnos([d], ds_text)[d])
            print(f"Falling back to BM25 results")
    
    except Exception as e:
        print(f"Error in SPLADE processing: {e}")
        # Fallback to BM25
        top_results = initial_results.head(10)
        top_results['text'] = top_results['docno'].map(lambda d: get_text_for_docnos([d], ds_text)[d])
        print(f"Falling back to BM25 results due to error")
    
    # Add query information to results
    top_results['qid'] = query_id
    top_results['query'] = query_text
    top_results['docid'] = top_results['docno']
    
    # Ensure rank is 0-based
    if 'rank' not in top_results.columns:
        top_results['rank'] = range(len(top_results))
    
    # Reorder columns to match MonoT5 CSV structure
    cols = ['qid', 'docid', 'docno', 'query', 'text', 'score', 'rank']
    top_results = top_results[cols]
    
    # Add to collection
    all_results.append(top_results)
    
    # Show progress for first query
    if idx == 0:
        print("\nSample results for first query:")
        print(top_results[['docno', 'score', 'text']].head(3))
    
    # Clean memory after each query
    gc.collect()

Processing query 1/43: do goldfish grow


msmarco-passage documents:  93%|█████████▎| 8266017/8841823 [00:26<00:01, 311359.52it/s]


SPLADE reranking successful

Sample results for first query:
      docno      score                                               text
26  8182159  27.926720  'Goldfish usually grow with their environment ...
10  1960257  27.720867  Goldfish Only Grow to the Size of Their Enclos...
24  2928707  27.577404  Goldfish Only Grow to the Size of Their Enclos...
Processing query 2/43: what is wifi vs bluetooth


msmarco-passage documents:  86%|████████▋ | 7637132/8841823 [00:22<00:03, 333207.59it/s]


SPLADE reranking successful
Processing query 3/43: why did the us volunterilay enter ww1


...

SPLADE reranking successful
Processing query 42/43: hydrogen is a liquid below what temperature


msmarco-passage documents:  99%|█████████▊| 8712732/8841823 [00:27<00:00, 312371.07it/s]


SPLADE reranking successful
Processing query 43/43: difference between a mcdouble and a double cheeseburger


msmarco-passage documents:  99%|█████████▉| 8795491/8841823 [00:27<00:00, 323218.53it/s]


SPLADE reranking successful


In [7]:
# Cell to save the SPLADE results to CSV
import pandas as pd

# Combine all results into a single DataFrame
combined_results = pd.concat(all_results) if all_results else pd.DataFrame()

# Save results to CSV
combined_results.to_csv('splade_results.csv', index=False)

print(f"\nRetrieved {len(combined_results)} documents across {len(topics)} queries")
print(f"Results saved to splade_results.csv")

# Show summary of results
print("\nTop documents for first few queries:")
print(combined_results.groupby('qid').head(1)[['qid', 'docno', 'score']].head(5))

# Print sample of CSV structure
print("\nSample of saved CSV structure:")
print(combined_results.head(3))


Retrieved 425 documents across 43 queries
Results saved to splade_results.csv

Top documents for first few queries:
        qid    docno      score
26   156493  8182159  27.926720
5   1110199  8160519  23.507820
8   1063750  4337526  21.525167
13   130510  8612903  24.929392
40   489204  1778458  19.598661

Sample of saved CSV structure:
       qid    docid    docno             query  \
26  156493  8182159  8182159  do goldfish grow   
10  156493  1960257  1960257  do goldfish grow   
24  156493  2928707  2928707  do goldfish grow   

                                                 text      score  rank  
26  'Goldfish usually grow with their environment ...  27.926720     0  
10  Goldfish Only Grow to the Size of Their Enclos...  27.720867     0  
24  Goldfish Only Grow to the Size of Their Enclos...  27.577404     1  
